In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection, metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from imblearn.over_sampling import SMOTE
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

xtrain = pd.read_csv("X_train.csv")
ytrain = pd.read_csv("Y_train.csv")
ytrain = ytrain.drop('id', axis = 1)
print(xtrain.shape)
sm = SMOTE(random_state=42)
x_tmp, y_tmp = sm.fit_resample(xtrain.values, ytrain.values)
xtrain = pd.DataFrame(x_tmp, columns = xtrain.columns)
ytrain = pd.DataFrame(y_tmp, columns = ytrain.columns)

xytrain = pd.concat([ytrain, xtrain], axis = 1).drop("id", axis = 1)

target = 'y'
#IDcol = 'ID'

xytrain.head()
print(xytrain.shape)

(4800, 1001)


/Users/andreasopedaleriksson/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(10800, 1001)


In [4]:
def modelfit(alg, dtrain, predictors, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        #xgb_param['num_class'] = 3
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds, 
                          metrics='mlogloss', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['y'],eval_metric='mlogloss')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("BMAC : %.4g" % metrics.balanced_accuracy_score(dtrain['y'].values, dtrain_predictions))
    #print("AUC Score (Train): %f" % metrics.balanced_accuracy_score(dtrain['y'], dtrain_predprob))
                    
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

In [5]:
#Choose all predictors except target & IDcols
predictors = [x for x in xytrain.columns if x not in [target]]
xgb1 = XGBClassifier(
 learning_rate =0.15,
 n_estimators=100,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 num_class = 3,
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, xytrain, predictors)


Model Report
BMAC : 0.9998


In [6]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', num_class = 3, nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='balanced_accuracy',n_jobs=4,iid=False, cv=5)
gsearch1.fit(xytrain[predictors],xytrain[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [ ]:
## Optimal params: {'max_depth': 7, 'min_child_weight': 5}

In [8]:
gsearch1.best_score_, gsearch1.best_params_, gsearch1.cv_results_

(0.9462037037037037,
 {'max_depth': 7, 'min_child_weight': 5},
 {'mean_fit_time': array([ 539.45602379,  549.57117367,  535.64925957,  779.42582097,
          738.48789458,  715.42313094, 4503.30701294, 2161.07991161,
         1021.99868579, 1240.68805761, 1034.32741885,  880.52029543]),
  'std_fit_time': array([1.75628040e+00, 1.18815684e+01, 2.27919738e+01, 2.10825342e+01,
         5.09489906e+00, 1.53672506e+01, 2.86613793e+03, 2.30126966e+03,
         3.73201897e+01, 1.82343480e+01, 1.47393974e+01, 8.48312977e+01]),
  'mean_score_time': array([0.58148122, 0.67184358, 0.62683125, 0.66654868, 0.68556409,
         0.58713908, 0.79203739, 0.75007458, 0.86818013, 0.71188173,
         0.49912047, 0.4001976 ]),
  'std_score_time': array([0.05948082, 0.10158132, 0.06625349, 0.06038848, 0.08891179,
         0.06003801, 0.08430719, 0.11541728, 0.11143985, 0.0904525 ,
         0.07349947, 0.14358151]),
  'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7, 9, 9, 9],
              

In [10]:
param_test2 = {
 'max_depth':[6,7,8],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.15, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', num_class = 3, nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='balanced_accuracy',n_jobs=4,iid=False, cv=5)
gsearch2.fit(xytrain[predictors],xytrain[target])
gsearch2.best_score_, gsearch2.best_params_, gsearch2.cv_results_

KeyboardInterrupt: 